# Generative Adversarial Networks (GAN)

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| Dennis G. Wilson | <a href="https://supaerodatascience.github.io/deep-learning/">https://supaerodatascience.github.io/deep-learning/</a>

adapted from the GAN class of Thomas Pierrot

<div class="alert alert-warning">
<b>Warning :</b>
This notebook is heavy on computational resources. Depending on your personal machine, it might be better to run it on <a href="https://colab.research.google.com/github/SupaeroDataScience/deep-learning/blob/main/GAN/Generative%20Adversarial%20Networks.ipynb">Colab</a>.
</div>

In [ ]:
%matplotlib inline
import itertools
import math
import time
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from IPython import display
from torch.autograd import Variable

### The goal of GANs

<b>Generative Adversarial Networks (GAN)</b> are a framework for generating new data which follows an existing distribution. This is often used with images is to generate images which look real, but which don't exist in the training set. The example we'll use today is the MNIST dataset, so we will generate new images which look like handwritten digits but which are created by a neural network.

<img src="img/distributions.png">

Goodfellow, Ian, et al. "Generative adversarial nets." Advances in neural information processing systems. 2014. [pdf](https://arxiv.org/pdf/1406.2661.pdf)

In this framework, two assumptions have to be made:
+ All the data from the training dataset is assumed to follow the same probability distribution noted $p_{data}$. In other words, if the MNIST database is considered, all the 28x28 matrices which represent hand-written numbers are assumed to follow the same probability distribution.

+ The database is assumed large enough that the probability distribution obtained represent well and only those data. In other words, if new sample can be generated from this distribution, they should look like the other elements of the database. In the case of the MNIST database, if new 28x28 matrices may be generated from the distribution $p_{data}$, then they should still represent hand-written numbers.

## Example: MNIST

We'll use the MNIST dataset for the exercise today. We'll download this dataset and make batching data loaders.

In [ ]:
# set batch_size
batch_size = 128

# the data, shuffled and split between train and test sets
# we normalize data to have values between -1 and 1
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]) ])

train_dataset = torchvision.datasets.MNIST(root='../data/', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
train_iterator = iter(train_loader)

As a reminder, let's look at some example digits from the MNIST set.

In [ ]:
num_test_samples = 16 # number of digits to plot

# create figure for plotting
size_figure_grid = int(math.sqrt(num_test_samples))
fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(6, 6))
for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
    ax[i,j].get_xaxis().set_visible(False)
    ax[i,j].get_yaxis().set_visible(False)

# load a batch of training data
images, labels = next(train_iterator)

# show a subpart of it
for k in range(num_test_samples):
    i = k//4
    j = k%4
    ax[i,j].cla()
    ax[i,j].imshow(images[k,:].data.cpu().numpy().reshape(28, 28), cmap='Greys')
    display.clear_output(wait=True)

The task for today is to learn to generate new images which look like these.

## The Generator

<img src="img/generator_schema.png">

One network - called the <b>Generator</b>- is built and is meant to transform vectors following a random probability distribution $p_z$ - called noise probability distribution - in vectors following natural data distribution $p_{data}$.

To follow the MNIST example, the generator will take as input vectors of real numbers following a gaussian distribution, $p_z$ and should output a 28x28 matrix whose values follow the data probability distribution followed by MNIST images, $p_{data}$. Hence, that network role would be to transform a gaussian distribution into a probability distribution represented by the data.

## The Discriminator

<img src="img/discriminator_schema.png">

To train the generator, we use a second network called the **Discriminator**. The role of this network is to learn the data distribution $p_{data}$ in order to classify if a sample comes from this distribution or not.

The <b>Discriminator</b>'s goal is to differentiate between samples following the real data probability distribution and samples generated by the generator. It takes as an input either a real sample or a sample output from the generator and is trained to return the likelihood of the image being real. In the maximum, the discriminator is expected to return 1 when it is given an image from the training dataset and to return 0 when it is given an image generated by the generator.

The **adversarial** aspect of the networks comes from the training methods. The generator is trained to fool the discriminator : ie to make it return 1 for generated samples. The networks play a 2-player minimax game. At the equilibrium, the discriminator should always return 0.5 : it cannot make the difference between real and fake samples anymore, hence the fake samples follow the same probability distribution than real ones. The generator is trained towards this result and ends up producing realistic results.

To get a better understanding, let us call G the generator, D the discriminator, $z$ a variable following the noise probability distribution $p_z$ and $x$ a variable following the data probability distribution $p_{data}$. Let us call as well $p_g$ the probability distribution followed by G’s outputs : $G(z)$. Then the generator's goal is to fool the discriminator returning real-like samples which means equalizing $p_g \approx p_{data}$.

The discriminator is trained to return $1$ for real data : $D(x) = 1$ and $0$ for fake data $D(G(z)) = 0$ while the generator is trained to make the discriminator returning $1$ as well for fake data. Hence, D and G play the two-player minimax game with value function $V(G, D)$:

$$\min_{G} \max_{D} V(D,G)  = \mathbb{E}_{x\sim p_{data}}[log D(x)] + \mathbb{E}_{z\sim p_{z}}[log(1 - D(G(z)))]$$

The implementation of this game results in the following algorithm, described in the original GAN paper:

<img src="img/gan_algo.png" style="max-width:100%; width: 70%">

# GANs in Practice


### Build the neural networks

We now build both the networks. In this notebook, as in the original paper, both will be simple fully connected network.

<img src="img/schema_gan_mnist.png" style="max-width:100%; width: 70%">


We'll start with a simple **Discriminator** model of feed-forward LeakyReLU units. We'll also add Dropout at each layer.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        out = self.model(x.view(x.size(0), 784))
        out = out.view(out.size(0), -1)
        return out

<div class="alert alert-success">
    <h2>Exercise 1</h2>

Write the generator code. It should take a vector of size 100 as input and output a flattened vector of 784 pixels. You can use feed-forward layers, similar to the Discriminator. What activation function should the final layer have? Consider that it is supposed to output black and white pixel values.
    
</div>

In [ ]:
# %load solutions/1_generator.py

Once the class are written, we instantiate the networks.

In [ ]:
discriminator = Discriminator()
generator = Generator()

Let's load some example data to go through the training steps. First we'll load the images and generate the labels. Note that the label for training the discriminator is not the number drawn in the image; it is simply 1 for all images from the real data set and 0 for all images from the generated set.

In [ ]:
images, _ = next(train_iterator)
images = Variable(images)
real_labels = Variable(torch.ones(images.size(0), 1))

To use the generator, we create $p_z$ following a normal distribution.

In [ ]:
# Sample from generator
noise = Variable(torch.randn(images.size(0), 100))
fake_images = generator(noise)
fake_labels = Variable(torch.zeros(images.size(0), 1))

Let's see what sort of images our random generator creates.

In [ ]:
plt.figure(figsize=(4,4))
for i in range(4):
    plt.subplot(2, 2, i+1)
    plt.imshow(fake_images[i,:].data.cpu().numpy().reshape(28, 28), cmap='Greys')
    plt.axis('off')

As could be expected, the random generator gives random noise images. We shouldn't expect much of a difference from the discriminator, given that its random. However, since the MNIST distribution $x$ and generated distribution $D(x)$ are so clearly different, even a random discriminitor should have a different output given these two inputs.

In [ ]:
d_real = discriminator(images)
d_fake = discriminator(fake_images)
d_real.detach().numpy()[:5], d_fake.detach().numpy()[:5]

We'll now train the discriminator and generator.

## GAN Training

The loss function we'll use in training is the [Binary Cross-Entropy loss](https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html#BCELoss). Looking at the PyTorch documentation, we can see that, by default, the `reduction` operator for $L$ is set to ``'mean'``, making the loss function:

$\ell(x, y) = L = \frac{1}{N}\sum\{l_1,\dots,l_N\}$

$l_n = - w_n \left[ y_n \cdot \log x_n + (1 - y_n) \cdot \log (1 - x_n) \right]$

In [ ]:
criterion = nn.BCELoss()
criterion(d_real, real_labels), criterion(d_fake, fake_labels)

This corresponds to the loss function defined above for the GAN:

$$\min_{G} \max_{D} V(D,G)  = \mathbb{E}_{x\sim p_{data}}[log D(x)] + \mathbb{E}_{z\sim p_{z}}[log(1 - D(G(z)))]$$

where for real data, $y_n$ is 1 and for generated data, $y_n$ is 0. We want the discriminator to predict 1 if the data is real and 0 if it is generated. This corresponds to the variables `real_labels` and `fake_labels`.

We'll also define the optimizers, simply using Adam for both the discriminiator and generator.

In [ ]:
lr = 0.0002
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))

<div class="alert alert-success">
    <h2>Exercise 2.</h2>

Write a training step for the discriminiator. The function definition is below. Fill in this missing components.
</div>

In [ ]:
def train_discriminator(discriminator, images, real_labels, fake_images, fake_labels):
    """
    Arguments:
        discriminator: discriminator model object
        images: a batch of data from the dataset
        real_labels: a vector of ones, size of images
        fake_images: a batch of images generated by the generator
        fake_labels: a vector of zeros, size of fake_images
    
    Returns:
        d_loss: discriminator loss
        real_output: output of the discriminator on the real images
        fake_output: output of the discrimiator on the fake images
    """    
    discriminator.zero_grad()
    # TODO: complete the code
    # d_loss =
    d_loss.backward()
    d_optimizer.step()
    return d_loss, real_output, fake_output

In [ ]:
# %load solutions/2_train_d.py

The second part of the min-max equation is the generator.

$$\min_{G} \max_{D} V(D,G)  = \mathbb{E}_{x\sim p_{data}}[log D(x)] + \mathbb{E}_{z\sim p_{z}}[log(1 - D(G(z)))]$$

Considering a static discriminator D, we have 

$$\min_{G} V(D,G)  = \mathbb{E}_{x\sim p_{data}}[log D(x)] + \mathbb{E}_{z\sim p_{z}}[log(1 - D(G(z)))]$$

However, the generator doesn't influence $\mathbb{E}_{x\sim p_{data}}[log D(x)]$. The generator's loss is therefore only

$$\min_{G} V(D,G)  = \mathbb{E}_{z\sim p_{z}}[log(1 - D(G(z)))]$$

Note that other loss metrics have been proposed for GANs to encourage more stable training. For example, at the beginning of training, the discriminator's job is easy and so it might train much faster than the generator, halting the adversarial training. In order to increase early gradients, it is proposed to maximize $log(D(G(z)))$ instead of minimizing $log(1-D(G(z)))$ for the generator.

Another loss function is using in the [Wasserstein GAN](https://arxiv.org/abs/1701.07875), which transforms the discriminator into a critic, outputting a real value which should be larger for real inputs than fake ones. The discriminator loss is then simply $D(x) - D(G(z))$ and the generator loss is $D(G(z))$.

<div class="alert alert-success">
    <h2>Exercise 3.</h2>

Write a training step for the generator using the cross entropy loss. The function definition is below. Fill in this missing components.
</div>

In [ ]:
def train_generator(generator, discriminator_outputs, real_labels):
    """
    Arguments:
        generator: generator model object
        discriminator_outputs: ouput of the discriminator on a set of values z, D(G(z))
        real_labels: a vector of ones, size of discriminator_outputs
        
    Returns:
        g_loss: generator loss
    """
    generator.zero_grad()
    #TODO: complete the code
    # g_loss =
    g_loss.backward()
    g_optimizer.step()
    return g_loss

In [ ]:
# %load solutions/3_train_g.py

# Model training

We'll now use our training functions in an iterative process over multiple epochs.

Due to the adversarial nature of GAN training, the different losses often oscillate over training instead of converging over time as we see in other deep learning training methods. As such, it is advised to frequently save results during training.

In [ ]:
# Prepare folder to store results
if not os.path.exists('results'):
    os.makedirs('results')
    
# Prepare folder to store models
if not os.path.exists('models'):
    os.makedirs('models')

In [ ]:
# Draw samples from the input distribution to inspect the generation on training 
num_test_samples = 16
test_noise = Variable(torch.randn(num_test_samples, 100))
# Set number of epochs and initialize figure counter
num_epochs = 4
num_batches = len(train_loader)
# Set counter
num_fig = 0

In [ ]:
# Create figure for plotting
size_figure_grid = int(math.sqrt(num_test_samples))
fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(6, 6))
for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
    ax[i,j].get_xaxis().set_visible(False)
    ax[i,j].get_yaxis().set_visible(False)

# Start training
t0 = time.time()
for epoch in range(num_epochs):
    for n, (images, _) in enumerate(train_loader):

        # Convert data to suitable format
        images = Variable(images)
        real_labels = Variable(torch.ones(images.size(0), 1))

        # Sample from generator
        noise = Variable(torch.randn(images.size(0), 100))
        fake_images = generator(noise)
        fake_labels = Variable(torch.zeros(images.size(0), 1))

        # Train the discriminator
        d_loss, real_score, fake_score = train_discriminator(
            discriminator, images, real_labels, fake_images, fake_labels)

        # Sample again from the generator and get output from discriminator
        noise = Variable(torch.randn(images.size(0), 100))
        fake_images = generator(noise)
        outputs = discriminator(fake_images)

        # Train the generator
        g_loss = train_generator(generator, outputs, real_labels)

        # Every half epoch generates pictures with to generator to monitor training
        if (n+1) % int(num_batches/2) == 0:
            # generate pictures
            test_images = generator(test_noise)
            # plot them
            for k in range(num_test_samples):
                i = k//4
                j = k%4
                ax[i,j].cla()
                ax[i,j].imshow(test_images[k,:].data.cpu().numpy().reshape(28, 28),
                               cmap='Greys')
            display.clear_output(wait=True)
            display.display(plt.gcf())

            # save the picture
            plt.savefig('results/mnist-gan-%03d.png'%num_fig)
            num_fig += 1
            # print log
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, g_loss: %.4f, '
                  'D(x): %.2f, D(G(z)): %.2f, time %.2f min'
                  % (epoch + 1, num_epochs, n+1, num_batches,
                     d_loss.detach().numpy(), g_loss.detach().numpy(),
                     real_score.detach().numpy().mean(),
                     fake_score.detach().numpy().mean(), (time.time()-t0)/60))

        # at the end of n epochs, save the models
        if epoch % 2 == 0:
            torch.save(generator.state_dict(),
                    os.path.join('models', 'generator.pkl'))
            torch.save(discriminator.state_dict(),
                    os.path.join('models', 'discriminator.pkl'))
